In [37]:
import psycopg2 as pg
from psycopg2 import OperationalError
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
warnings.warn('DelftStack')
warnings.warn('Do not show this message')

In [2]:
connection = pg.connect(dbname = 'postgres',
                       host = 'localhost',
                       user = 'postgres',
                       password = 'admin',
                       port = 5432)
cursor = connection.cursor()

In [60]:
connection.rollback()

In [68]:
with open('hw1_diagram.sql', 'r') as f:
    script = f.read()
try:   
    cursor.execute(script)
except OperationalError as e:
    print(e)

In [28]:
print(script)

CREATE TABLE "customer" (
  "customer_id" integer PRIMARY KEY,
  "first_name" varchar,
  "last_name" varchar,
  "gender" varchar,
  "DOB" date,
  "job_id" integer,
  "wealth_segment" varchar,
  "deceased_indicator" varchar,
  "owns_car" varchar,
  "address_id" integer,
  "property_valuation" int,

  UNIQUE("customer_id")
);

CREATE TABLE "transaction" (
  transaction_id integer PRIMARY KEY,
  "product_id" integer,
  "customer_id" integer REFERENCES customer("customer_id"),
  "transaction_date" date,
  "online_order" bool,
  "order_status" varchar,
  UNIQUE("transaction_id")
);

CREATE TABLE "products" (
  "product_id" integer PRIMARY KEY,
  "brand" varchar,
  "product_line" varchar,
  "product_class" varchar,
  "product_size" varchar,
  "list_price" float,
  "standard_cost" float,

  UNIQUE(product_id)
);

CREATE TABLE "addresses" (
  "address_id" integer PRIMARY KEY,
  "address" text,
  "postcode" integer,
  "state" varchar,
  "country" varchar,
  UNIQUE("address_id")
);

CREATE TABLE

In [68]:
transaction = pd.read_excel("tables_data.xlsx", sheet_name = 'transaction')
customer = pd.read_excel("tables_data.xlsx", sheet_name = 'customer')

## После приведения БД к 3НФ получилось 5 таблиц
Ниже по тексту я объясняю почему получилось именно так.
<img src="images/db_model.png" width="520" height="500">

## Заполняю таблицы *transaction* и *products*

Когда transaction была в 1НФ(изначальная таблица), то транзакция(transaction_id) имела несколько транзитивно зависимых(transaction -> product -> атрибуты продукта) от него атрибутов: 'brand','product_line', 'product_class', 'product_size', 'list_price', 'standard_cost', которые в свою очередь не зависели от транзакции(transaction_id), а зависели от product, поэтому я вынес их в отдельную таблицу products с зависимостью теперь только от продукта. Теперь таблица transaction в 3НФ.

При заполнении буду использовать первые 50 строк.

___

Я обнаружил, что разным по сути товарам присваивается один и тот же product_id. Это нужно исправить, сделав для каждого уникального по характеристикам велосипеда уникальный идентификатор product_id и также в таблице transaction поменять эти product_id на новые.

Например, product_id == 12, разные велосипеды(Giant Bicycles и WeareA2B) имеют одинаковый идентификатор:

In [121]:
transaction.iloc[:50].query("product_id == 12")

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,new_product_id
9,10,12,3262,2017-08-30,True,Approved,WeareA2B,Standard,medium,medium,1231.15,161.60,92
14,15,12,247,2017-06-11,False,Approved,Giant Bicycles,Standard,medium,large,1765.30,709.48,9
38,39,12,427,2017-09-12,False,Approved,WeareA2B,Standard,medium,medium,1231.15,161.60,92


In [69]:
transaction['new_product_id'] = transaction.groupby(['brand', 'product_line', 'product_class', 'product_size', 'list_price', 'standard_cost']).ngroup()


## Проверяю, уникален ли id для каждого уникального товара

Теперь уникальные по характеристикам модели от Giant Bicycles и WeareA2B имеют разные id.

In [70]:
transaction[['product_id','new_product_id','brand','product_line','product_class','product_size','list_price','standard_cost']].drop_duplicates().iloc[:50].query("product_id == 12")


,product_id,new_product_id,brand,product_line,product_class,product_size,list_price,standard_cost
9,12,92,WeareA2B,Standard,medium,medium,1231.15,161.60
14,12,9,Giant Bicycles,Standard,medium,large,1765.30,709.48


## Вставляю правила в transaction

In [71]:
transaction_script = 'insert into transaction values '
def insert_row(row):
    global transaction_script
    transaction_script = transaction_script + f"({row['transaction_id']},{row['product_id']},{row['customer_id']},'{row['transaction_date']}',{row['online_order']},'{row['order_status']}'),\n"
transaction.iloc[:50].apply(insert_row, axis = 1)
transaction_script = transaction_script[:-2]

In [93]:
print(transaction_script)

insert into transaction values (1,2,2950,'2017-02-25 00:00:00',False,'Approved'),
(2,3,3120,'2017-05-21 00:00:00',True,'Approved'),
(3,37,402,'2017-10-16 00:00:00',False,'Approved'),
(4,88,3135,'2017-08-31 00:00:00',False,'Approved'),
(5,78,787,'2017-10-01 00:00:00',True,'Approved'),
(6,25,2339,'2017-03-08 00:00:00',True,'Approved'),
(7,22,1542,'2017-04-21 00:00:00',True,'Approved'),
(8,15,2459,'2017-07-15 00:00:00',False,'Approved'),
(9,67,1305,'2017-08-10 00:00:00',False,'Approved'),
(10,12,3262,'2017-08-30 00:00:00',True,'Approved'),
(11,5,1986,'2017-01-17 00:00:00',False,'Approved'),
(12,61,2783,'2017-01-05 00:00:00',True,'Approved'),
(13,35,1243,'2017-02-26 00:00:00',True,'Approved'),
(14,16,2717,'2017-09-10 00:00:00',False,'Approved'),
(15,12,247,'2017-06-11 00:00:00',False,'Approved'),
(16,3,2961,'2017-10-10 00:00:00',False,'Approved'),
(17,79,2426,'2017-04-03 00:00:00',False,'Approved'),
(18,33,1842,'2017-06-02 00:00:00',False,'Approved'),
(19,54,2268,'2017-04-06 00:00:00',True

In [72]:
try:   
    cursor.execute(transaction_script)
except OperationalError as e:
    print(e)

## Вставляю правила в products

In [73]:
products_script = 'insert into products values '
def insert_row(row):
    global products_script
    products_script = products_script + f"({row['new_product_id']},'{row['brand']}','{row['product_line']}','{row['product_class']}','{row['product_size']}',{row['list_price']},{row['standard_cost']}),\n"
transaction[['new_product_id','brand','product_line','product_class','product_size','list_price','standard_cost']].drop_duplicates().iloc[:50].apply(insert_row, axis = 1)
products_script = products_script[:-2]

In [74]:
try:   
    cursor.execute(products_script)
except OperationalError as e:
    print(e)

In [134]:
print(products_script)

insert into products values (61,'Solex','Standard','medium','medium',71.49,53.62),
(82,'Trek Bicycles','Standard','medium','large',2091.47,388.92),
(41,'OHM Cycles','Standard','low','medium',1793.43,248.82),
(28,'Norco Bicycles','Standard','medium','medium',1198.46,381.1),
(9,'Giant Bicycles','Standard','medium','large',1765.3,709.48),
(2,'Giant Bicycles','Road','medium','medium',1538.99,829.65),
(88,'WeareA2B','Standard','medium','medium',60.34,45.26),
(93,'WeareA2B','Standard','medium','medium',1292.84,13.44),
(60,'Solex','Standard','medium','large',1071.23,380.74),
(92,'WeareA2B','Standard','medium','medium',1231.15,161.6),
(72,'Trek Bicycles','Mountain','low','medium',574.64,459.71),
(40,'OHM Cycles','Standard','low','medium',71.16,56.93),
(80,'Trek Bicycles','Standard','low','medium',1057.51,154.4),
(23,'Norco Bicycles','Standard','high','small',1661.92,1479.11),
(29,'Norco Bicycles','Standard','medium','medium',1555.58,818.01),
(14,'Giant Bicycles','Standard','medium','small',131

# Заполняю таблицы customer, jobs, addresses

Я выделил несколько отдельных сущностей из customer: jobs, adresses. Теперь все атрибуты в этих трех таблицах нетранзитивно зависят от первичного ключа и БД находится в 3НФ.

<img src="images/db_model.png" width="520" height="500">

In [10]:
# Подготовка таблиц customer и jobs
customer.columns
jobs = customer[['job_title','job_industry_category']].drop_duplicates()
jobs['job_id'] = range(jobs.shape[0])
customer = customer.merge(jobs,
              how = 'inner',
              on = ['job_title','job_industry_category'])
customer = customer.drop(['job_title','job_industry_category'], axis = 1)
# Подготовка таблицы addresses
addresses = customer[['address', 'postcode', 'state', 'country']].drop_duplicates()
addresses['address_id'] = range(addresses.shape[0])
customer = customer.merge(addresses,
              how = 'inner',
              on = ['address', 'postcode', 'state', 'country'])
customer = customer.drop(['address', 'postcode', 'state', 'country'], axis = 1)
customer[['job_id','address_id']].head(3)

,job_id,address_id
0,0,0
1,0,1
2,0,2


## Вставляю правила в customer

In [31]:
customer['last_name'] = customer['last_name'].str.replace(r"'", '`')

In [47]:
customer_script = 'insert into customer values '
def insert_row(row):
    global customer_script
    if pd.isnull(row['last_name']):
        row['last_name'] = 'NULL'
        customer_script = customer_script + f'''({row['customer_id']},'{row['first_name']}',{row['last_name']},'{row['gender']}','{row['DOB']}',{row['job_id']},'{row['wealth_segment']}','{row['deceased_indicator']}','{row['owns_car']}',{row['address_id']},{row['property_valuation']}),\n'''
        return None
    if pd.isnull(row['DOB']):
        row['DOB'] = 'NULL'
        customer_script = customer_script + f'''({row['customer_id']},'{row['first_name']}','{row['last_name']}','{row['gender']}',{row['DOB']},{row['job_id']},'{row['wealth_segment']}','{row['deceased_indicator']}','{row['owns_car']}',{row['address_id']},{row['property_valuation']}),\n'''
        return None
    customer_script = customer_script + f'''({row['customer_id']},'{row['first_name']}','{row['last_name']}','{row['gender']}','{row['DOB']}',{row['job_id']},'{row['wealth_segment']}','{row['deceased_indicator']}','{row['owns_car']}',{row['address_id']},{row['property_valuation']}),\n'''
    
customer.iloc[:50].apply(insert_row, axis = 1)
customer_script = customer_script[:-2]

In [48]:
print(customer_script)

insert into customer values (1,'Laraine','Medendorp','F','1953-10-12 00:00:00',0,'Mass Customer','N','Yes',0,10),
(409,'Tatiania','O`Kane','Female','1978-01-23 00:00:00',0,'Mass Customer','N','Yes',1,9),
(949,'Jarid','Grebner','Male','1975-09-07 00:00:00',0,'Mass Customer','N','Yes',2,1),
(2,'Eli','Bockman','Male','1980-12-16 00:00:00',1,'Mass Customer','N','Yes',3,10),
(387,'Spike','Thieme','Male','1977-02-19 00:00:00',1,'Mass Customer','N','No',4,2),
(1524,'Bonni','Drakers','Female','1975-04-29 00:00:00',1,'Mass Customer','N','Yes',5,10),
(2726,'Bradford','Monkton','Male','1976-03-18 00:00:00',1,'Affluent Customer','N','No',6,7),
(2727,'Kim','Gabbitis','Female','1988-07-22 00:00:00',1,'High Net Worth','N','Yes',7,10),
(2740,'Godiva','Bulward','Female','1963-02-02 00:00:00',1,'Affluent Customer','N','Yes',8,1),
(3033,'Danielle','Kener','Female','1979-03-17 00:00:00',1,'High Net Worth','N','No',9,11),
(3270,'Tracey','Redsull','Male','1957-04-12 00:00:00',1,'Mass Customer','N','Yes',10,

In [50]:
try:   
    cursor.execute(customer_script)
except OperationalError as e:
    print(e)

## Вставляю правила в jobs

In [53]:
jobs

,job_title,job_industry_category,job_id
0,Executive Secretary,Health,0
1,Administrative Officer,Financial Services,1
2,Recruiting Manager,Property,2
3,NaN,IT,3
4,Senior Editor,NaN,4
...,...,...,...
3953,Human Resources Assistant III,Financial Services,1044
3954,Geologist II,Argiculture,1045
3961,Project Manager,Argiculture,1046
3984,Web Developer IV,Property,1047


In [57]:
jobs_script = 'insert into jobs values '
def insert_row(row):
    global jobs_script
    jobs_script = jobs_script + f'''({row['job_id']},'{row['job_title']}','{row['job_industry_category']}'),\n'''
    
jobs.apply(insert_row, axis = 1)
jobs_script = jobs_script[:-2]

In [58]:
print(jobs_script)

insert into jobs values (0,'Executive Secretary','Health'),
(1,'Administrative Officer','Financial Services'),
(2,'Recruiting Manager','Property'),
(3,'nan','IT'),
(4,'Senior Editor','nan'),
(5,'nan','Retail'),
(6,'nan','Financial Services'),
(7,'Media Manager I','nan'),
(8,'Business Systems Development Analyst','Argiculture'),
(9,'Senior Quality Engineer','Financial Services'),
(10,'nan','Property'),
(11,'Nuclear Power Engineer','Manufacturing'),
(12,'Developer I','Financial Services'),
(13,'Account Executive','Financial Services'),
(14,'Junior Executive','Manufacturing'),
(15,'Media Manager IV','nan'),
(16,'Sales Associate','nan'),
(17,'Professor','nan'),
(18,'Geological Engineer','Manufacturing'),
(19,'Project Manager','Manufacturing'),
(20,'Safety Technician I','Manufacturing'),
(21,'nan','nan'),
(22,'Research Assistant I','Argiculture'),
(23,'Accounting Assistant III','Financial Services'),
(24,'Editor','Financial Services'),
(25,'Research Nurse','Health'),
(26,'Senior Quality Eng

In [61]:
try:   
    cursor.execute(jobs_script)
except OperationalError as e:
    print(e)

## Вставляю правила в addresses

In [62]:
addresses.columns

Index(['address', 'postcode', 'state', 'country', 'address_id'], dtype='object')

In [63]:
addresses_script = 'insert into addresses values '
def insert_row(row):
    global addresses_script
    addresses_script = addresses_script + f'''({row['address_id']},'{row['address']}',{row['postcode']},'{row['state']}','{row['country']}'),\n'''
    
addresses.apply(insert_row, axis = 1)
addresses_script = addresses_script[:-2]

In [64]:
print(addresses_script)

insert into addresses values (0,'060 Morning Avenue',2016,'New South Wales','Australia'),
(1,'17 Sunfield Road',2211,'NSW','Australia'),
(2,'97 Aberg Pass',4720,'QLD','Australia'),
(3,'6 Meadow Vale Court',2153,'New South Wales','Australia'),
(4,'9457 Arizona Plaza',2642,'NSW','Australia'),
(5,'28 Mcguire Junction',2171,'NSW','Australia'),
(6,'63 International Drive',4165,'QLD','Australia'),
(7,'04 Schmedeman Court',2196,'NSW','Australia'),
(8,'33297 Hoepker Way',2843,'NSW','Australia'),
(9,'01654 Milwaukee Plaza',2000,'NSW','Australia'),
(10,'6030 South Drive',2120,'NSW','Australia'),
(11,'0 Holy Cross Court',4211,'QLD','Australia'),
(12,'726 Warrior Center',2077,'NSW','Australia'),
(13,'17979 Del Mar Point',2448,'New South Wales','Australia'),
(14,'52 Carey Alley',4740,'QLD','Australia'),
(15,'833 Luster Way',4005,'QLD','Australia'),
(16,'163 Straubel Trail',2217,'New South Wales','Australia'),
(17,'6262 Elgar Avenue',4735,'QLD','Australia'),
(18,'05 Butternut Crossing',4552,'QLD','A

In [65]:
try:   
    cursor.execute(addresses_script)
except OperationalError as e:
    print(e)

In [75]:
connection.commit()

In [76]:
cursor.close()
connection.close()